In [12]:
from IPython.display import display, HTML,clear_output
# css = """
# .output {
# flex-direction: row;
# flex-wrap: wrap;
# }
# """

# HTML('<style>{}</style>'.format(css))

ERROR! Session/line number was not unique in database. History logging moved to new session 187


In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
# %pip install --no-cache-dir matplotlib

# from pandasql import sqldf
import requests
import brotli
from datetime import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [15]:
pd.options.mode.copy_on_write = True

In [16]:
origin_url="https://www.nseindia.com/market-data/most-active-contracts"
url = 'https://www.nseindia.com/api/snapshot-derivatives-equity?index=contracts'
f_url = 'https://www.nseindia.com/api/quote-derivative?symbol=NIFTY'
default_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

header = {
             "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
             "Accept-Language": "en-US,en;q=0.9,hi;q=0.8"
            }


session = requests.session()
nse_live = session.get(origin_url, headers=default_header)
cookies = nse_live.cookies
session.get(url, headers=header, cookies=cookies)


<Response [200]>

In [17]:
def index_data(f_data):
    f_df = [item for item in f_data if item.get("metadata", {}).get("optionType") == '-']
    met_df = pd.DataFrame(f_df)
    df = pd.concat([met_df.drop(columns=['metadata','marketDeptOrderBook']),met_df["metadata"].apply(pd.Series),met_df["marketDeptOrderBook"].apply(pd.Series)],axis=1)
    c_df = pd.concat([df['expiryDate'],df['bid'].apply(pd.Series),df['ask'].apply(pd.Series)],axis=1)
    df = pd.concat([df.drop(columns=['carryOfCost','tradeInfo','otherInfo','bid','ask']),df['carryOfCost'].apply(pd.Series),df['tradeInfo'].apply(pd.Series),df['otherInfo'].apply(pd.Series)],axis=1)
    df = pd.concat([df.drop(columns='price'),df['price'].apply(pd.Series)],axis=1)
    df = df.drop(columns=['instrumentType','optionType','identifier','openPrice','highPrice','lowPrice','closePrice',
                    'prevClose','change','pChange','numberOfContractsTraded','totalTurnover','instrumentType','carry','lastPrice','marketLot','premiumTurnover',"impliedVolatility","clientWisePositionLimits","marketWidePositionLimits"
          ,"tickSize"])
    return df,c_df

In [18]:
chain_origin_url="https://www.nseindia.com/option-chain"
ni_default_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}
nifty_url ='https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
nifty_url='https://www.nseindia.com/api/option-chain-v3?type=Indices&symbol=NIFTY&expiry=17-Jul-2025'
ni_header = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/111.0.0.0 Safari/537.36",
    "Sec-Fetch-User": "?1", "Accept": "*/*", "Sec-Fetch-Site": "none", "Sec-Fetch-Mode": "navigate",
    "Accept-Encoding": "gzip, deflate, br", "Accept-Language": "en-US,en;q=0.9,hi;q=0.8"
    }
ni_session = requests.Session()
ni_nse_live = session.get(chain_origin_url, headers=ni_default_header)
ni_cookies = ni_nse_live.cookies
# nse_live = session.get("http://nseindia.com", headers=ni_header)
ni_session.get(nifty_url, headers=ni_header,cookies=ni_cookies)

<Response [200]>

In [19]:
columns = [
    "underlying", "expiryDate", "optionType", "strikePrice", "lastPrice", "numberOfContractsTraded",
    "totalTurnover", "openInterest", "underlyingValue", "pChange", "timestamp"
]
# Create an empty DataFrame
filter_df = pd.DataFrame(columns=columns)

In [20]:
def dataframe(df):
    data = []
    for i in range(0,len(df)):
        calloi=callcoi=cltp=putoi=putcoi=pltp=0
        ctotaltradevaluem=ctotalBellq=ctotalSellq=ptotaltradevaluem=ptotalBellq=ptotalSellq=0
        pbidQty=pbidprice=paskQty=paskPrice=cbidQty=cbidprice=caskQty=caskPrice=CALL_iv=PUT_iv = 0
        stp = df['strikePrice'][i]
        if(df['CE'][i]==0):
            calloi = callcoi =0
        else:
            calloi = float(df['CE'][i]['openInterest'])
            callcoi = float(df['CE'][i]['changeinOpenInterest'])
            ctotalBellq = float(df['CE'][i]['totalBuyQuantity'])
            ctotalSellq = float(df['CE'][i]['totalSellQuantity'])
            # CE_BP1 = float(df['CE'][i]['buyPrice1'])
            # CE_Bq1 = float(df['CE'][i]['buyQuantity1'])
            # CE_SP1 = float(df['CE'][i]['sellPrice1'])
            # CE_Sq1 = float(df['CE'][i]['sellQuantity1'])
            cstp = float(df['CE'][i]['underlyingValue'])
        if(df['PE'][i]==0):
            putoi = putcoi = 0
        else:
            putoi = float(df['PE'][i]['openInterest'])
            putcoi = float(df['PE'][i]['changeinOpenInterest'])
            ptotalBellq = float(df['PE'][i]['totalBuyQuantity'])
            ptotalSellq = float(df['PE'][i]['totalSellQuantity'])
            # PE_BP1 = float(df['PE'][i]['buyPrice1'])
            # PE_Bq1 = float(df['PE'][i]['buyQuantity1'])
            # PE_SP1 = float(df['PE'][i]['sellPrice1'])
            # PE_Sq1 = float(df['PE'][i]['sellQuantity1'])


        opdata = {
            'CALL_OI' : calloi,'CALL_CHNG_OI':callcoi,
            # 'CE_BP1':CE_BP1,'CE_Bq1':CE_Bq1,'CE_SP1':CE_SP1,'CE_Sq1':CE_Sq1,
            'CALL_BQ':ctotalBellq,'CALL_SA':ctotalSellq,'strikePrice':stp,
            'PUT_SA':ptotalSellq,'PUT_BQ':ptotalBellq,'PUT_CHNG_OI':putcoi,'PUT_OI' : putoi,
            # 'PE_BP1':PE_BP1,'PE_Bq1':PE_Bq1,'PE_SP1':PE_SP1,'PE_Sq1':PE_Sq1,
            'current_stp':cstp}
        data.append(opdata)
    optionchain = pd.DataFrame(data)
    optionchain['%_SQ'] = ((optionchain['CALL_SA'] - optionchain['PUT_SA']) / ((optionchain['CALL_SA'] + optionchain['PUT_SA']) / 2)) * 100
    optionchain['%_BQ'] = ((optionchain['CALL_BQ'] - optionchain['PUT_BQ']) / ((optionchain['CALL_BQ'] + optionchain['PUT_BQ']) / 2)) * 100
    #optionchain =optionchain.append(optionchain[['CALL_OI','CALL_CHNG_OI','PUT_OI','PUT_CHNG_OI','CALL_TTV','PUT_TTV','CALL_BQ','PUT_BQ','CALL_SA','PUT_SA']].sum(),ignore_index=True).fillna(0)
    return optionchain


In [21]:
def plot(df):
    df = df.drop_duplicates().tail(20)
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # Ensure timestamp is datetime

    fig, ax = plt.subplots(figsize=(10,6))

    # Group by expiryDate and plot each group
    for key, grp in df.groupby(['expiryDate']):
        ax.plot(grp['timestamp'], grp['lastPrice'], linestyle='-', marker='o', label=f'Last Price ({key})')
        ax.plot(grp['timestamp'], grp['underlyingValue'], linestyle='--', marker='x', label=f'Underlying Value ({key})')

    # Adding labels and title
    ax.set_xlabel('Timestamp')
    ax.set_ylabel('Value')
    ax.set_title('Last Price and Underlying Value over Time by Expiry Date')

    # Adding legend
    ax.legend(loc='best', fontsize='small')

    # Adding grid and rotating x-axis labels
    ax.grid(True)
    plt.xticks(rotation=45)

    plt.tight_layout()

    # Save the plot as an image file
    plt.savefig('line_graph.png')

    # Show the plot
    plt.show()


In [22]:
def process_nifty_options(df, option_df):
    """
    Processes NIFTY options data by performing a left join with API data, 
    filtering relevant columns, and renaming option-specific columns.

    Parameters:
    df (pd.DataFrame): The main dataframe containing options data.
    ni_session (requests.Session): Session object for API requests.
    nifty_url (str): URL to fetch NIFTY option data.
    ni_header (dict): Headers for API request.

    Returns:
    pd.DataFrame: Processed dataframe with relevant columns.
    """

    # Filter for NIFTY and sort data
    df = df[df['underlying'] == 'NIFTY'].sort_values(by=['strikePrice', 'openInterest'], ascending=True)

    # Perform a left join
    join_df = df.merge(option_df, on="strikePrice", how="left")

    # Define common columns
    common_cols = [
        "expiryDate", "optionType", "strikePrice", "lastPrice", 
        "numberOfContractsTraded", "totalTurnover", "openInterest", 
        "underlyingValue", "pChange", "current_stp"
    ]

    # Conditional selection of call/put columns
    call_cols = ["CALL_OI", "CALL_CHNG_OI", "CALL_BQ", "CALL_SA"]
    put_cols = ["PUT_OI", "PUT_CHNG_OI", "PUT_BQ" ,"PUT_SA" ]

    # Rename mapping
    rename_mapping = {
        "CALL_OI": "oi", "CALL_CHNG_OI": "coi", "CALL_BQ": "buy_q", "CALL_SA": "sell_q",
        "PUT_OI": "oi", "PUT_CHNG_OI": "coi", "PUT_BQ": "buy_q", "PUT_SA": "sell_q"
    }

    # Create new columns based on 'optionType'
    for call_col, put_col in zip(call_cols, put_cols):
        join_df[rename_mapping[call_col]] = np.where(
            join_df["optionType"].str.lower() == "call", join_df[call_col], join_df[put_col]
        )

    # Keep only relevant columns
    final_cols = common_cols + ["oi", "coi", "buy_q", "sell_q"]
    join_df = join_df[final_cols]
    join_df.drop(columns =['totalTurnover'],inplace=True)
    return join_df

In [22]:
def main_dis_run(filter_df):
  try:
    while True:
      clear_output(wait=True)
      data = session.get(url,headers=header).json()['value']['data']
      df = pd.DataFrame(data)
      df.drop(columns =['identifier','instrumentType','instrument','premiumTurnover'],inplace=True)
      f_data = session.get(f_url,headers=header).json()['stocks']
      c_df = index_data(f_data=f_data)
      try:
        ni_data = ni_session.get(nifty_url, headers=ni_header, cookies=ni_cookies).json()['filtered']['data']
        df1 = pd.DataFrame(ni_data).fillna(0)
        option_df = dataframe(df1)
        price = int(option_df[option_df['current_stp'] > 0]['current_stp'].unique())
        display(process_nifty_options(df, option_df))
        # display(f_df[f_df['underlying'] == 'NIFTY'].sort_values(by=['strikePrice', 'openInterest'], ascending=True))
        display(c_df[0],c_df[1])
        range_price = list(range(round(price/100)*100-250,round(price/100)*100+250,50))
        display(option_df[(option_df.strikePrice.isin(range_price))])
      except Exception as e:
        print(f'issue with option chain')
        # raise e
        dis_df = df[df['underlying'] == 'NIFTY'].sort_values(by=['strikePrice', 'openInterest'], ascending=True)
        dis_df.drop(columns =['totalTurnover'],inplace=True)
        display(dis_df)
        display(c_df[0],c_df[1])
      
      filtered_df = df[(df['underlying'] == 'NIFTY') & (df['optionType'] == '-')] 
      filtered_df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S') 
      date = datetime.now().strftime('%Y-%m-%d')
      # csv_file_path = f'D:/badri/OptionChainData/nifty/{date}.csv'
      csv_file_path = f'C:/Users/2199222/OneDrive - Cognizant/Desktop/optiondata/nifty/{date}.csv'
      # filtered_df.to_csv(csv_file_path, mode='a', header=False, index=False)
      filter_df = pd.concat([filter_df, filtered_df], ignore_index=True)
      # df[df['underlying'] == 'NIFTY'].sort_values(by=['strikePrice','openInterest'],ascending=True).to_csv('D:/badri/OptionChainData/nifty/nifty.csv',index=False)
      display(df[df['underlying'] != 'NIFTY'].sort_values(by=['lastPrice', 'openInterest'], ascending=True))
      # display(filter_df)
      plot(filter_df)
      time.sleep(5)
  except Exception as e:
      clear_output(wait=True)
      print(f'trying to retry to re-slove issue')
      # raise e
      main_dis_run(filter_df)

main_dis_run(filter_df)

trying to retry to re-slove issue


In [ ]:
f_data = session.get(f_url,headers=header).json()['stocks']

In [ ]:
f_data

[{'metadata': {'instrumentType': 'Index Options',
   'expiryDate': '10-Jul-2025',
   'optionType': 'Put',
   'strikePrice': 25500,
   'identifier': 'OPTIDXNIFTY10-07-2025PE25500.00',
   'openPrice': 63.9,
   'highPrice': 79.85,
   'lowPrice': 58.05,
   'closePrice': 0,
   'prevClose': 68.35,
   'lastPrice': 73.95,
   'change': 5.6000000000000085,
   'pChange': 8.193123628383333,
   'numberOfContractsTraded': 324266,
   'totalTurnover': 17272.03},
  'underlyingValue': 25491,
  'volumeFreezeQuantity': 1801,
  'marketDeptOrderBook': {'totalBuyQuantity': 4744350,
   'totalSellQuantity': 1187700,
   'bid': [{'price': 73.8, 'quantity': 975},
    {'price': 73.75, 'quantity': 2550},
    {'price': 73.7, 'quantity': 1950},
    {'price': 73.65, 'quantity': 3450},
    {'price': 73.6, 'quantity': 1800}],
   'ask': [{'price': 73.95, 'quantity': 225},
    {'price': 74, 'quantity': 2550},
    {'price': 74.05, 'quantity': 3825},
    {'price': 74.1, 'quantity': 1950},
    {'price': 74.15, 'quantity': 12

In [ ]:
# df = pd.read_csv('D:/badri/OptionChainData/nifty/nifty.csv')
# df_unique = df.drop_duplicates(subset=df.columns.difference(['timestamp']), keep='last')